In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

#Helper function from the NN

def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav
#160ms and 3kz

def load_wav_for_map(filename, label,indice):
  return load_wav_16k_mono(filename), label,indices

#tkt
def create_data_from_path(path, length,label):
    dataset = tf.data.Dataset.from_tensor_slices(np.arange(length))
    return dataset.map(lambda index: (path + str(index), label))

def extract_embedding(wav_data, label,indice):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings),
            tf.repeat(indice, num_embeddings))

yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

nb_of_samples_per_class = 65
path_to_deformed = "./samples_A_DEFORMED1_C0/sample_A_DEFORMED1_C0"
path_to_RX = "./samples_A_RX1_C0/sample_A_RX1_C0"


vfuncRx = np.vectorize(lambda index: (path_to_RX + str(index)+ ".wav"))
vfuncDeformed = np.vectorize(lambda index: (path_to_deformed + str(index)+ ".wav"))
indices = np.arange(nb_of_samples_per_class)

filenamesRX = vfuncRx(indices)
filenames_deformed = vfuncRx(indices)

#we assign label 0 to RX and 1 to deformed
labelsRX = np.zeros(nb_of_samples_per_class) 
labelsDeformed = np.ones(nb_of_samples_per_class)

filenames = np.concatenate((filenamesRX, filenames_deformed))
labels = np.concatenate((labelsRX, labelsDeformed))
#random values added in order split the data later
rand = np.random.rand(len(filenames)) * 10
dataRX = tf.data.Dataset.from_tensor_slices((filenames, labels,rand))

dataRX = dataRX.map(load_wav_for_map)

dataRX = dataRX.map(extract_embedding).unbatch()

#Split the data
cached_data = dataRX.cache()
dataRX.cache()

train_ds = cached_data.filter(lambda embedding, label, rand: rand < 2)
other = cached_data.filter(lambda embedding, label, rand: rand >=7)
test_ds = other.filter(lambda embedding, label, rand: rand <= 8)
val_ds = other.filter(lambda embedding, label, rand: rand > 8)

remove_indice_column = lambda embedding, label, indice: (embedding, label)

train_ds = train_ds.map(remove_indice_column)
val_ds = val_ds.map(remove_indice_column)
test_ds = test_ds.map(remove_indice_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)


In [2]:
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 525,826
Trainable params: 525,826
Non-trainable params: 0
_________________________________________________________________


In [3]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

print()

In [6]:
history = my_model.fit(train_ds,
                       epochs=2,
                       #validation_data=val_ds,
                       callbacks=callback)

Epoch 1/2


InvalidArgumentError:  Input element must have the same batch size in each component. Component 0 had size 63 but component 2 had size, 4095.
	 [[node IteratorGetNext
 (defined at /home/fermeli/.local/lib/python3.8/site-packages/keras/engine/training.py:866)
]] [Op:__inference_train_function_16391]

Errors may have originated from an input operation.
Input Source operations connected to node IteratorGetNext:
In[0] iterator (defined at /home/fermeli/.local/lib/python3.8/site-packages/keras/engine/training.py:1216)

Operation defined at: (most recent call last)
>>>   File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/usr/lib/python3/dist-packages/traitlets/config/application.py"